In [7]:
def md_simulation(cif_file, temperature=300, n_steps=5000, time_step=1.0):
    from pymatgen.core import Structure
    from chgnet.model import StructOptimizer
    from pymatgen.core import Structure
    from chgnet.model.dynamics import MolecularDynamics
    from chgnet.model import CHGNet
    from pymatgen.io.ase import AseAtomsAdaptor
    from tf_chpvk_pv.config import CRYSTALLM_DATA_DIR
    import os

    cif_file_name = cif_file.split('/')[-1]

    chemical_formula = cif_file_name.replace('_1.cif', '')

    relaxed_folder = CRYSTALLM_DATA_DIR / "relaxed_cif_files"

    
    print(chemical_formula)
    structure = Structure.from_file(cif_file)


    relaxed_cif_file_name = cif_file_name.replace(".cif", "_relaxed.cif")
    relaxed_cif_file = os.path.join(relaxed_folder, relaxed_cif_file_name)
    
    md_cif_file_name = cif_file_name.replace(".cif", "_md_relaxed.cif")
    md_cif_file = os.path.join(relaxed_folder, md_cif_file_name)
    
    # Initialize the structural optimizer
    relaxer = StructOptimizer()

    # Perform the relaxation
    result = relaxer.relax(structure, verbose=False)

    # Access the results
    final_structure = result["final_structure"]
    final_energy = result['trajectory'].energies[-1]
    print(f"Relaxed Total Energy: {final_energy:.4f} eV")
    final_structure.to(relaxed_cif_file)

    md = MolecularDynamics(
        atoms=final_structure,  # Fixed: changed from 'structure' to 'atoms'
        model=CHGNet.load(),
        ensemble="nvt",        # Added: specify the thermodynamic ensemble
        temperature=temperature,       # K
        timestep=time_step,          # fs
    )

    md.run(n_steps) # Number of steps is passed to run()

    # The current state of the atoms after md.run()
    final_ase_atoms = md.atoms 

    # Convert back to Pymatgen to check stability/symmetry
    final_structure = AseAtomsAdaptor.get_structure(final_ase_atoms)

    # Check the final energy
    final_energy = final_ase_atoms.get_potential_energy()
    print(f"Final Potential Energy: {final_energy:.4f} eV")
    
    final_structure.to(md_cif_file)

In [11]:
def detection_imaginary_modes(cif_file):
    from pymatgen.core import Structure
    from chgnet.model import StructOptimizer
    from pymatgen.core import Structure
    from chgnet.model import CHGNet
    from chgnet.model.dynamics import CHGNetCalculator
    from phonopy import Phonopy
    import numpy as np
    from tf_chpvk_pv.config import CRYSTALLM_DATA_DIR
    import os

    cif_file_name = cif_file.split('/')[-1]

    chemical_formula = cif_file_name.replace('_1.cif', '')

    relaxed_folder = CRYSTALLM_DATA_DIR / "relaxed_cif_files"

    
    print(chemical_formula)
    structure = Structure.from_file(cif_file)

    relaxed_cif_file_name = cif_file_name.replace(".cif", "_relaxed.cif")
    relaxed_cif_file = os.path.join(relaxed_folder, relaxed_cif_file_name)

    # Initialize the structural optimizer
    relaxer = StructOptimizer()

    # Perform the relaxation
    result = relaxer.relax(structure, verbose=False)

    # Access the results
    final_structure = result["final_structure"]
    final_energy = result['trajectory'].energies[-1]
    print(f"Relaxed Total Energy: {final_energy:.4f} eV")
    final_structure.to(relaxed_cif_file)



    # 1. Load the model and set up the calculator
    model = CHGNet.load()
    calc = CHGNetCalculator(model)

    # 2. Initialize Phonopy (assuming 'relaxed_structure' is a Pymatgen structure)
    # We convert Pymatgen structure to a format Phonopy understands
    from pymatgen.io.phonopy import get_phonopy_structure
    phonopy_structure = get_phonopy_structure(final_structure)

    phonon = Phonopy(phonopy_structure, supercell_matrix=[[2, 0, 0], [0, 2, 0], [0, 0, 2]])

    # 3. Generate displacements
    phonon.generate_displacements(distance=0.01)

    # Fix for version 2.46.1:
    displaced_scs = phonon.supercells_with_displacements 

    set_of_forces = []
    for sc in displaced_scs:
        # Convert phonopy atoms to pymatgen structure
        from pymatgen.io.phonopy import get_pmg_structure
        pmg_sc = get_pmg_structure(sc)
        
        # Run CHGNet prediction
        prediction = model.predict_structure(pmg_sc)
        
        # Extract forces (ensure they are in the shape [N_atoms, 3])
        set_of_forces.append(prediction['f'])

    # 4. Produce phonon frequencies
    phonon.produce_force_constants(forces=set_of_forces)
    phonon.run_mesh([10, 10, 10])  # Define q-point mesh
    dict_results = phonon.get_mesh_dict()
    frequencies = dict_results['frequencies']

    # Check imaginary modes (negative frequencies)
    message = ''
    imaginary_frequencies = []
    if (frequencies < -0.01).any():
        message = "Dynamically unstable: Imaginary modes detected"
        print("Imaginary frequencies (in THz):", frequencies[frequencies < -0.01])
        imaginary_frequencies.append(frequencies[frequencies < -0.01])
    else:
        message = "Dynamically stable"

    print(message)

    return {'formula': chemical_formula,
            'message': message,
            'imaginary_frequencies':imaginary_frequencies}


In [8]:
import os
from tf_chpvk_pv.config import CRYSTALLM_DATA_DIR

folder_path = CRYSTALLM_DATA_DIR / "cif_files"

files = os.listdir(folder_path)

# Filter for files only
files = [os.path.join(folder_path, f) for f in files]

for cif_file in files:
    md_simulation(cif_file)

SmLuS3
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda
Relaxed Total Energy: -133.6646 eV
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda
CHGNet will run on cuda
NVT-Berendsen-MD created


/home/dagar/anaconda3/envs/synthesis_planning/lib/python3.10/site-packages/ase/md/nvtberendsen.py:98: RuntimeWarning: divide by zero encountered in scalar divide
  1.0 + (self.temperature / old_temperature - 1.0) * tautscl


Final Potential Energy: -132.8282 eV
DyInS3
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda
Relaxed Total Energy: -110.5016 eV
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda
CHGNet will run on cuda
NVT-Berendsen-MD created
Final Potential Energy: -110.1259 eV
EuUS3
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda
Relaxed Total Energy: -175.4671 eV
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda
CHGNet will run on cuda
NVT-Berendsen-MD created
Final Potential Energy: -174.4599 eV
CeCdS3
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda
Relaxed Total Energy: -106.8881 eV
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda
CHGNet will run on cuda
NVT-Berendsen-MD created
Final Potential Energy: -106.1579 eV
SmZrS3
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda
Relaxed Total Energy: -144.7101 eV
CHGNet v0.3.0 initia

In [12]:
list_of_dicts = []

for cif_file in files:
    dict_file = detection_imaginary_modes(cif_file)
    list_of_dicts.append(dict_file)

SmLuS3
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda
Relaxed Total Energy: -133.6646 eV
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda
CHGNet will run on cuda
Imaginary frequencies (in THz): [-0.81257414 -0.50602869 -0.79106962 -0.44149658 -0.75071532 -0.37242573
 -0.69663102 -0.3925558  -0.24432569 -0.14333491 -0.63396988 -0.53712172
 -0.39214714 -0.36337225 -0.70902172 -0.69315736 -0.66409887 -0.62722104
 -0.34534271 -0.58710518 -0.51165622 -0.48069504 -0.48321371 -0.48436023
 -0.48614751 -0.32999315 -0.4852201  -0.44792438 -0.11994655 -0.26494047
 -0.1408279  -0.29979455 -0.25717068 -0.73292948 -0.54432296 -0.22084864
 -0.73089279 -0.46454988 -0.10267934 -0.72390858 -0.35607994 -0.08858181
 -0.71496288 -0.41057715 -0.30008134 -0.05179903 -0.69296216 -0.61108559
 -0.29794141 -0.23022071 -0.63985241 -0.63286241 -0.6287181  -0.62997628
 -0.37268533 -0.61923642 -0.54925882 -0.44482954 -0.44003952 -0.45088762
 -0.4742669  -0.25938361

In [15]:
import pandas as pd

df = pd.DataFrame(data=list_of_dicts)

df.to_csv(CRYSTALLM_DATA_DIR / "dynamical_stability_results.csv", index=False)

In [46]:
import numpy as np

temps = np.linspace(100, 1000, 10)  # K
# 1. First, tell phonopy to calculate the thermal properties
phonon.run_thermal_properties(temperatures=temps)

# 2. Extract the results from the dictionary
t_props = phonon.get_thermal_properties_dict()
free_energies = t_props['free_energy']

# 3. Now you can loop as intended
for T, F in zip(temps, free_energies):
    print(f"T = {T:4.0f} K | F_vib = {F:.3f} eV")


T =  100 K | F_vib = 27.490 eV
T =  200 K | F_vib = -26.664 eV
T =  300 K | F_vib = -104.281 eV
T =  400 K | F_vib = -198.102 eV
T =  500 K | F_vib = -304.211 eV
T =  600 K | F_vib = -420.198 eV
T =  700 K | F_vib = -544.438 eV
T =  800 K | F_vib = -675.763 eV
T =  900 K | F_vib = -813.293 eV
T = 1000 K | F_vib = -956.343 eV
